<a href="https://colab.research.google.com/github/axjasf/Personal-Finance-Categorizer/blob/main/Bank_files_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About

* This project is meant to bring all my personal finance related transactions into one easy to understand view.

* Mechanism
    * It reads CSV files from several US and German banks and Credit Card processors and harmonizes them into one dataframe.
    * It converts currency into USD.
* How to:
    * Review last sync date here: Dec-31, 2023
    * Download
        * Chase: rename to chase.csv
        * Wells Checking: rename to wellsfargo_checking.csv
        * Commerzbank
            * rename to commerzbank.csv
            * Adjust dates in file through removing dates that are too early or too late.
        * Apple:
            * Wallet -> Apple Card -> Card Balance -> Statement X -> Download
            * Use ChatGPT to combine these into one file
            * Double check file to avoid repetition of header row
            * rename to apple.csv
    * Import EUR - USD Exchange Rate
        * Download: https://www.wsj.com/market-data/quotes/fx/EURUSD/historical-prices
        * Open Eur-USD Exchange Rate CSV: https://drive.google.com/drive/folders/1ZuICF3FiMtcW5_ofhVR0MKv10Y0JJZEq?usp=drive_link
        * Append to Sheet and check for consistency
        * Re-Upload into Google Drive
    * Run Bank files Conversion
        * Adjust TRANSACTIONS_PATH = HOME_PATH + "transactions/2023/"
        * Run Bank files conversion.ipynb
        * Check transactions.csv for consistency

# Setup

## Paths

In [19]:
# Path settings
HOME_PATH = "/content/drive/MyDrive/Colab Notebooks/budget/"
CONFIG_PATH = HOME_PATH + "config/"
TRANSACTIONS_PATH = HOME_PATH + "transactions/2023/"

## Loading of Libraries

In [20]:
import json
import pandas as pd
import numpy as np
import os

## Define global Variables

In [21]:
# Define the transactions dataframe and load the JSON configuration for the different banks
bank_transactions = {}

bank_files = {
        "Chase": "chase.csv",
        "Wells Fargo Checking": "wellsfargo_checking.csv",
        "Apple": "apple.csv",
        "Commerzbank": "commerzbank.csv"
    }

config_files = {
    "Exchange Rates EUR USD": 'eur_usd_exchange_rates.csv',
}

all_transactions_file = "transactions.csv"

# If we are using Google Drive, prefix each value in the dictionary with the ..._PATH variable
bank_files = {key: f"{TRANSACTIONS_PATH}{value}" for key, value in bank_files.items()}
config_files = {key: f"{CONFIG_PATH}{value}" for key, value in config_files.items()}

# Define overall transactions dataframe
all_transactions = []

# File Conversion

* For each bank file:
    * Load file into individual df
    * Basic quality control on the individual df level
    * Transform columns into target columns
        * Add Bank ID field as well as numberical ID field
    * Add individual df to transactions df

* Special transformations for non-US banks:
    * Date conversion
    * EUR to USD conversion based on an existing file (date and exchange rate or an API call to a free service)

In [22]:
def quality_control(df):
    missing_values = df.isnull().sum()
    column_data_types = df.dtypes

    return missing_values, column_data_types

In [23]:
def adjust_field_names(df, bank=""):

    if 'Category' in df.columns:
        df = df.rename(columns={"Category" : "oldCategory"})

    df.insert(4, 'SplitID',"")
    df.insert(0, 'Date','')
    df.insert(1, 'Payee','')
    df.insert(2, 'Category Type','')
    df.insert(3, 'Category','')
    df.insert(4, 'chkPayee','')
    df.insert(5, 'chkCategory','')
    df.insert(6, 'chkSplit','')
    df.insert(7, 'chkEURUSD','')

    return df

## Wells Fargo

### Wells Fargo Checking

In [24]:
# Read the Wells Fargo Checking CSV
bank = 'Wells Fargo Checking'
bank_transactions[bank] = pd.read_csv(bank_files[bank], header=None, names=["Transaction Date", "Amount (USD)", "Status", "Memo", "Description"])

# Adjust field names (if any specific adjustments are required)

# Convert 'Transaction Date' column to datetime
bank_transactions[bank]['Date'] = pd.to_datetime(bank_transactions[bank]['Transaction Date'], errors='coerce')

# Check for problematic dates (rows where the date conversion failed)
problematic_dates = bank_transactions[bank][bank_transactions[bank]['Date'].isna()]

# Perform quality control checks
missing_values, column_data_types = quality_control(bank_transactions[bank])

# Final touches for Wells Fargo only
bank_transactions[bank] = bank_transactions[bank].drop(columns=['Transaction Date', 'Status', 'Memo'])  # Assuming 'Status' is not needed, adjust as necessary
#bank_transactions[bank]['Amount (USD)'] *= -1

## Chase

In [25]:
bank = 'Chase'
bank_transactions[bank] = pd.read_csv(bank_files[bank])

bank_transactions[bank] = adjust_field_names(bank_transactions[bank])

bank_transactions[bank]['Date'] = pd.to_datetime(bank_transactions[bank]['Transaction Date'], errors='coerce')
problematic_dates = bank_transactions[bank][bank_transactions[bank]['Date'].isna()]
missing_values, column_data_types = quality_control(bank_transactions[bank])

bank_transactions[bank] = bank_transactions[bank].drop(columns=['Post Date', 'oldCategory', 'Type', 'Memo', 'Transaction Date'])
bank_transactions[bank] = bank_transactions[bank].rename(columns={"Amount" : "Amount (USD)"})


## Apple

In [26]:
bank = 'Apple'
bank_transactions[bank] = pd.read_csv(bank_files[bank])

bank_transactions[bank] = adjust_field_names(bank_transactions[bank], bank)

bank_transactions[bank]['Date'] = pd.to_datetime(bank_transactions[bank]['Transaction Date'], errors='coerce')
problematic_dates = bank_transactions[bank][bank_transactions[bank]['Date'].isna()]
missing_values, column_data_types = quality_control(bank_transactions[bank])

# Final touches for Apple Card only
bank_transactions[bank] = bank_transactions[bank].drop(columns=['Transaction Date', 'Clearing Date', 'Merchant', 'oldCategory', 'Type', 'Purchased By'])
bank_transactions[bank]['Amount (USD)'] *= -1

## Commerzbank

In [27]:
bank = 'Commerzbank'
bank_transactions[bank] = pd.read_csv(bank_files[bank])

bank_transactions[bank] = adjust_field_names(bank_transactions[bank])


bank_transactions[bank]['Date'] = pd.to_datetime(bank_transactions[bank]['Transaction date'], errors='coerce', format='%d.%m.%Y') # For Commerzbank, Day.Month.Year
problematic_dates = bank_transactions[bank][bank_transactions[bank]['Date'].isna()]
missing_values, column_data_types = quality_control(bank_transactions[bank])

bank_transactions[bank] = bank_transactions[bank][bank_transactions[bank]['Amount'] != 0]


bank_transactions[bank].insert(7, "Amount (USD)","")

bank_transactions[bank] = bank_transactions[bank].rename(columns={"Booking text" : "Description"})

### EUR to USD conversion
# https://www.wsj.com/market-data/quotes/fx/EURUSD/historical-prices

exchange_rates_data = pd.read_csv(config_files['Exchange Rates EUR USD'])

# Convert the date columns to consistent datetime format
exchange_rates_data['Date'] = pd.to_datetime(exchange_rates_data['Date'], format='%m/%d/%Y')

# Merge on the date columns to add the exchange rate to bank_transactions[bank]
bank_transactions[bank] = bank_transactions[bank].merge(exchange_rates_data[['Date', 'Close']], on='Date', how='left')

# Add the chkEURUSD column based on the ' Close' column value
bank_transactions[bank]['chkEURUSD'] = np.where(bank_transactions[bank]['Close'].isna(), 'E', 'A')

# Convert the Amount from EUR to USD
bank_transactions[bank]['Amount (USD)'] = bank_transactions[bank]['Amount'] * bank_transactions[bank]['Close']

# Drop the ' Close' column as it's not needed anymore in bank_transactions[bank]
bank_transactions[bank].drop('Close', axis=1, inplace=True)

### End of currency conversion

#bank_transactions[bank].drop(bank_transactions[bank].columns[[16, 15, 14, 13, 12, 10, 9, 8]], axis=1, inplace=True)
bank_transactions[bank] = bank_transactions[bank].drop(columns=['Transaction date', 'Value date', 'Transaction type', 'Amount', 'Account of initiator', 'Bank code of account of initiator', 'IBAN of account of initiator'])


# File concatenation

In [28]:
# Concatenation of the transactional dataframes
all_transactions = pd.concat(bank_transactions, keys=bank_transactions.keys())
all_transactions['Account-ID'] = all_transactions.index.get_level_values(0) + "-" + all_transactions.index.get_level_values(1).astype(str)

# Load the "data" sheet from the personal_finance.xlsx file
master_data = pd.read_excel(HOME_PATH + "personal_finance.xlsx", sheet_name="data")

# Extract account names and ID numbers
master_data['Account'] = master_data['Account-ID'].str.split('-').str[0]
master_data['ID'] = master_data['Account-ID'].str.split('-').str[1].astype(int)
highest_ids = master_data.groupby('Account')['ID'].max()

# Update the all_transactions index
for account in all_transactions['Account-ID'].str.split('-').str[0].unique():
    if account in highest_ids:
        start_id = highest_ids[account] + 1
        mask = all_transactions['Account-ID'].str.startswith(account)
        count = mask.sum()
        new_ids = ["{}-{}".format(account, i) for i in range(start_id, start_id + count)]
        all_transactions.loc[mask, 'Account-ID'] = new_ids

# Reset the index of all_transactions
all_transactions.reset_index(drop=True, inplace=True)

# Output generation

## Dataframe preparation

In [29]:
# Reorder Columns

print(all_transactions.columns)

all_transactions = all_transactions[[
    'Date',
    'Account-ID',
    'SplitID',
    'Payee',
    'Category Type',
    'Category',
    'Amount (USD)',
    'Description',
    'chkPayee',
    'chkCategory',
    'chkEURUSD']]

# Sort rows
all_transactions = all_transactions.sort_values(by=['Date', 'Account-ID', 'SplitID'], ascending=[False, True, True])

# Formating
all_transactions['Amount (USD)'] = all_transactions['Amount (USD)'].round(2)
#all_transactions['Amount (USD)'] = all_transactions['Amount (USD)'].apply(lambda x: "${:,.2f}".format(x))


Index(['Amount (USD)', 'Description', 'Date', 'Payee', 'Category Type',
       'Category', 'chkPayee', 'chkCategory', 'chkSplit', 'chkEURUSD',
       'SplitID', 'Currency', 'Account-ID'],
      dtype='object')


## Output file generation

In [30]:
print(len(all_transactions))

# Export both files into Google Drive
all_transactions.to_csv(f"{TRANSACTIONS_PATH}{all_transactions_file}", index=False)

872
